In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn import linear_model

class CB(object):
    def __init__(self,item, rate):
        user = rate['user_id']
        user = pd.DataFrame(user.unique())
        
        self.user = user
        self.item = item
        self.rate = rate.values
        self.item.set_index('name', inplace = True)
        indices = pd.Series(self.item.index)
        # self.X0 = self.item.values
        # self.X_train_counts = self.X0[:,-19:]
        self.n_user = user.shape[0]

    def convert(self):
        
        transformer = TfidfVectorizer()
        self.tfidf = transformer.fit_transform(self.item['final']).toarray()

        # transformer = TfidfTransformer(smooth_idf = True, norm ='l2')
        # self.tfidf = transformer.fit_transform((self.X_train_counts).tolist()).toarray()
        return self.tfidf

    def get_items_rated_by_user(self, rate_matrix, user_id):
        y = rate_matrix[:,0]
        ids = np.where(y == user_id + 1)[0]
        items_id = rate_matrix[ids,1]
        scores = rate_matrix[ids,2]

        return (items_id, scores)

    def calculate(self):
        tfidf = self.convert()
        d = (tfidf).shape[1] # data dimension
        W = np.zeros((d, self.n_user))
        b = np.zeros((1, self.n_user))

        for n in range(self.n_user):    
            ids, scores = self.get_items_rated_by_user(self.rate, n)
            clf = Ridge(alpha=0.01, fit_intercept  = True)
            Xhat = tfidf[ids, :]
            
            clf.fit(Xhat, scores) 
            W[:, n] = clf.coef_
            b[0, n] = clf.intercept_
        return (W, b)

    def fit(self):
        self.W, self.b = self.calculate()
        self.Yhat = np.dot(self.tfidf, self.W) + self.b
        return self.Yhat
    
    def predict(self, user_id, rate_test):
        ids, scores = self.get_items_rated_by_user(rate_test, user_id)
        np.set_printoptions(precision=4)
        print ('Rated movies ids:', ids + 1 )
        print ('True ratings:', scores)
        print ('Predicted ratings:', self.Yhat[ids, user_id])
    
    def evaluate(self, rates):
        se = 0
        cnt = 0
        Yhat = self.Yhat
        for n in range(n_user):
            ids, scores_truth = get_items_rated_by_user(rates, n)
            scores_pred = Yhat[ids, n]
            e = scores_truth - scores_pred 
            se += (e*e).sum(axis = 0)
            cnt += e.size 
        return np.sqrt(se/cnt)

    # function that takes in movie title as input and returns the top 10 recommended movies
    def recommendations_item(self, user_id, top = 10):
        
        recommended_items = []
        # # gettin the index of the movie that matches the title
        # idx = indices[indices == title].index[0]

        # creating a Series with the similarity scores in descending order
        score_series = pd.Series(self.Yhat[:, user_id]).sort_values(ascending = False)

        # getting the indexes of the 10 most similar movies
        top_10_indexes = list(score_series.iloc[0:top].index)
        
        # populating the list with the titles of the best 10 matching movies
        for i in top_10_indexes:
            recommended_items.append(list((self.item).index)[i])
            
        return recommended_items

    def recommend(self, top = 10):
        recommender = pd.DataFrame()
        for n in range(self.n_user):
            recommend_items = self.recommendations_item(n, top)
            recommender[n+1] = recommend_items
        return recommender

In [ ]:
rate = pd.read_csv('/content/drive/MyDrive/Training AI/crawl/crawl to csv/at_user.csv')


In [ ]:
user = rate['user_id']
user = pd.DataFrame(user.unique())

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Training AI/crawl/crawl to csv/audio_tech.csv')
item = df[['name','final']]

In [ ]:
test = CB(item,rate)
Yhat = test.fit()

In [ ]:
test.recommend(5)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000
0,Smile Jamaica In Ear Headphones,Combo Wooden iPhone Charging Dock,Stir It Up Turntable,PowerKnit USB-C to Lightning Charging Cable,Recycled Bamboo Chopsticks Tablet Stand,TerraCycle E-Waste Zero Waste Box,PowerKnit USB-C to USB-C Cable,Smile Jamaica In Ear Headphones,No Bounds Sport Waterproof Bluetooth Speaker,Apollo Single Wireless Charging Pad,Organic Cotton Laptop Sleeve 15 in,Cotopaxi Del Día Quince Laptop Sleeve 15in,TerraCycle Alkaline Batteries Zero Waste Box,No Bounds Waterproof Bluetooth Speaker,Apollo Single Wireless Charging Pad,Champ Lite 5K Portable Charger,Combo Wooden iPhone Charging Dock,Hemp Tablet Bag,Smile Jamaica In Ear Headphones,Triple Wooden iPhone Charging Dock,Recycled Bamboo Chopsticks Tablet Stand,Uplift 2 Bluetooth In Ear Headphones,Compostable iPhone Pela Case,Wooden Laptop Stand,USB-C Wall Charger,Compostable iPhone Pela Case,Champ 10K Portable Charger,Compostable iPhone Pela Case,Nimble Disc Clear iPhone Case,Cotopaxi Del Día Quince Laptop Sleeve 15in,USB-C to USB-C Cable,Organic Cotton Laptop Sleeve 15 in,Triple Wooden iPhone Charging Dock,Cotopaxi Del Día Quince Laptop Sleeve 15in,Little Bird In Ear Headphones,Little Bird In Ear Headphones,TerraCycle Alkaline Batteries Zero Waste Box,No Bounds Waterproof Bluetooth Speaker,No Bounds Waterproof Bluetooth Speaker,TerraCycle E-Waste Zero Waste Box,...,Compostable iPhone Pela Case,TerraCycle E-Waste Zero Waste Box,USB-C Wall Charger,Compostable iPhone Pela Case,TerraCycle Alkaline Batteries Zero Waste Box,Felt &#038; Cork Desk Mat,Cotopaxi Del Día Trece Laptop Sleeve 13in,USB-C to USB-C Cable,Wooden Phone Wireless Charger,PowerKnit USB-C to Lightning Charging Cable,Wooden Headphone Stand,Compostable iPhone Pela Case,TerraCycle E-Waste Zero Waste Box,WALLY Wall Charger,TerraCycle Alkaline Batteries Zero Waste Box,No Bounds Sport Waterproof Bluetooth Speaker,No Bounds Sport Waterproof Bluetooth Speaker,TerraCycle Alkaline Batteries Zero Waste Box,Compostable iPhone Pela Case,Stir It Up Turntable,Positive Vibration 2 Bluetooth On-Ear Headphones,Smile Jamaica In Ear Headphones,Compostable iPhone Pela Case,Champ Lite 5K Portable Charger,Little Bird In Ear Headphones,Uplift 2 Bluetooth In Ear Headphones,Hemp Tablet Bag,No Bounds Waterproof Bluetooth Speaker,Positive Vibration 2 Bluetooth On-Ear Headphones,Uplift 2 Bluetooth In Ear Headphones,TerraCycle E-Waste Zero Waste Box,TerraCycle E-Waste Zero Waste Box,Little Bird In Ear Headphones,Stir It Up Turntable,Get Together Mini Bluetooth Speaker,TerraCycle Alkaline Batteries Zero Waste Box,Organic Cotton Laptop Sleeve 15 in,Zero Waste Liquid Screen Protector,PowerKnit USB-C to Lightning Charging Cable,Solo Wireless Charging Pad
1,Positive Vibration 2 Bluetooth On-Ear Headphones,Geometric Wooden Phone Charging Dock,Ethos Radiation Reduction Insert,Nimble Disc Clear iPhone Case,TerraCycle Alkaline Batteries Zero Waste Box,TerraCycle Alkaline Batteries Zero Waste Box,Nimble Disc Clear iPhone Case,Positive Vibration 2 Bluetooth On-Ear Headphones,No Bounds Waterproof Bluetooth Speaker,Little Bird In Ear Headphones,Organic Cotton Laptop Sleeve 13in,Cotopaxi Del Día Trece Laptop Sleeve 13in,PowerKnit USB-C to USB-C Cable,No Bounds Sport Waterproof Bluetooth Speaker,Dual Wireless Charging Pad,PowerKnit USB-C to Lightning Charging Cable,Slim Wooden Phone Wireless Charger,Cotopaxi Del Día Quince Laptop Sleeve 15in,Positive Vibration 2 Bluetooth On-Ear Headphones,Geometric Wooden Phone Charging Dock,Uplift 2 Bluetooth In Ear Headphones,Organic Cotton Laptop Sleeve 15 in,Ethos Radiation Reduction Insert,Organizer Wooden iPhone Charging Dock,USB-C Car Charger,Ethos Radiation Reduction Insert,PowerKnit USB-C to Lightning Charging Cable,Ethos Radi